<a href="https://colab.research.google.com/github/autowiki4/Deep-Chem-Tutorials/blob/main/examples/tutorials/Putting_Multitask_Learning_to_Work.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Putting Multitask Learning to Work

This notebook walks through the creation of multitask models on MUV [1]. The goal is to demonstrate how multitask methods can provide improved performance in situations with little or very unbalanced data.

## Colab

This tutorial and the rest in this sequence are designed to be done in Google colab. If you'd like to open this notebook in colab, you can use the following link.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/deepchem/deepchem/blob/master/examples/tutorials/Putting_Multitask_Learning_to_Work.ipynb)



In [1]:
!pip install --pre deepchem
import deepchem
deepchem.__version__

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.
Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead
wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


'2.8.1.dev'

The MUV dataset is a challenging benchmark in molecular design that consists of 17 different "targets" where there are only a few "active" compounds per target. There are 93,087 compounds in total, yet no task has more than 30 active compounds, and many have even less. Training a model with such a small number of positive examples is very challenging.  Multitask models address this by training a single model that predicts all the different targets at once. If a feature is useful for predicting one task, it often is useful for predicting several other tasks as well. Each added task makes it easier to learn important features, which improves performance on other tasks [2].

To get started, let's load the MUV dataset.  The MoleculeNet loader function automatically splits it into training, validation, and test sets.  Because there are so few positive examples, we use stratified splitting to ensure the test set has enough of them to evaluate.

In [2]:
import deepchem as dc
import numpy as np

tasks, datasets, transformers = dc.molnet.load_muv(split='stratified')
train_dataset, valid_dataset, test_dataset = datasets

Streaming output truncated to the last 5000 lines.
[22:09:37] DEPRECATION WARNING: please use MorganGenerator
[22:09:37] DEPRECATION WARNING: please use MorganGenerator
[22:09:37] DEPRECATION WARNING: please use MorganGenerator
[22:09:37] DEPRECATION WARNING: please use MorganGenerator
[22:09:37] DEPRECATION WARNING: please use MorganGenerator
[22:09:37] DEPRECATION WARNING: please use MorganGenerator
[22:09:37] DEPRECATION WARNING: please use MorganGenerator
[22:09:37] DEPRECATION WARNING: please use MorganGenerator
[22:09:37] DEPRECATION WARNING: please use MorganGenerator
[22:09:37] DEPRECATION WARNING: please use MorganGenerator
[22:09:37] DEPRECATION WARNING: please use MorganGenerator
[22:09:37] DEPRECATION WARNING: please use MorganGenerator
[22:09:37] DEPRECATION WARNING: please use MorganGenerator
[22:09:37] DEPRECATION WARNING: please use MorganGenerator
[22:09:37] DEPRECATION WARNING: please use MorganGenerator
[22:09:37] DEPRECATION WARNING: please use MorganGenerator
[22:0

Now let's train a model on it.  We'll use a MultitaskClassifier, which is a simple stack of fully connected layers.

In [3]:
n_tasks = len(tasks)
n_features = train_dataset.get_data_shape()[0]
model = dc.models.MultitaskClassifier(n_tasks, n_features)
model.fit(train_dataset)

0.11549418449401855

Let's see how well it does on the test set.  We loop over the 17 tasks and compute the ROC AUC for each one.

In [4]:
y_true = test_dataset.y
y_pred = model.predict(test_dataset)
metric = dc.metrics.roc_auc_score
for i in range(n_tasks):
    score = metric(dc.metrics.to_one_hot(y_true[:,i]), y_pred[:,i])
    print(tasks[i], score)

MUV-466 0.5429518180189862
MUV-548 0.7979133082572094
MUV-600 0.5440981551137382
MUV-644 0.6961669353394232
MUV-652 0.7953430055525703
MUV-689 0.664651620992298
MUV-692 0.3277449399964177
MUV-712 0.8391635321511732
MUV-713 0.4882590005373455
MUV-733 0.7480476446355007
MUV-737 0.6268762314168009
MUV-810 0.961615618842916
MUV-832 0.9532867633888591
MUV-846 0.9601647859573705
MUV-852 0.7371484864768045
MUV-858 0.7219595199713416
MUV-859 0.41735170857511283


Not bad!  Recall that random guessing would produce a ROC AUC score of 0.5, and a perfect predictor would score 1.0.  Most of the tasks did much better than random guessing, and many of them are above 0.9.

# Congratulations! Time to join the Community!

Congratulations on completing this tutorial notebook! If you enjoyed working through the tutorial, and want to continue working with DeepChem, we encourage you to finish the rest of the tutorials in this series. You can also help the DeepChem community in the following ways:

## Star DeepChem on [GitHub](https://github.com/deepchem/deepchem)
This helps build awareness of the DeepChem project and the tools for open source drug discovery that we're trying to build.

## Join the DeepChem Gitter
The DeepChem [Gitter](https://gitter.im/deepchem/Lobby) hosts a number of scientists, developers, and enthusiasts interested in deep learning for the life sciences. Join the conversation!

# Bibliography

[1] https://pubs.acs.org/doi/10.1021/ci8002649

[2] https://pubs.acs.org/doi/abs/10.1021/acs.jcim.7b00146